In [1]:
import pickle

import gensim.downloader
import numpy as np
import pandas as pd
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kevinhass/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
embed = gensim.downloader.load("glove-wiki-gigaword-200")

In [3]:
def get_glove_embedding(word):
    return embed[word]

In [4]:
def get_similarity(v1, v2):
    numerator = np.dot(v1, v2)
    denominator = np.linalg.norm(v1) * np.linalg.norm(v2)
    similarity = numerator / denominator
    return similarity

In [5]:
def find_glove_similar_word(answer, guess, num):
    res = embed.closer_than(answer, guess)
    if res:
        most_similar_key = res[int(len(res)/2)]
        return(True, most_similar_key)
    else:
        return(False, "N/A")

In [6]:
from openai import OpenAI
client = OpenAI(
    api_key=""
)

def get_GPT_embedding(word):
    response = client.embeddings.create(
        input=word,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding

In [7]:
def preprocess(input):
    return input.strip().lower()

In [8]:
import random
def secret_word_engine():
    while(True):
        category = input("Choose a category from: 1. actions, 2. adjectives, 3. food, 4. intangible objects, 5. fantasy")
        match category:
            case "1":
                return random.choice(["tokens", "embeddings", "model", "transformers"]) #NLP
            case "2":
                return random.choice(["exam", "pencil", "book", "friends"]) #school
            case "3":
                return random.choice(["cake", "chocolate", "pie", "bread"]) #food
            case "4":
                return random.choice(["teacher", "engineer", "driver", "soldier"]) #careers
            case "5":
                return random.choice(["tennis", "football", "skiing", "basketball"]) #sports
            #for user testing
            case "6":
                return "cake"
            case "7":
                return "book"
            case "8":
                return "basketball"
            case _:
                print("Enter a valid number for the category")

In [9]:
def find_hypernyms(word, guess, num):
    # Get all synsets for the word
    synsets = wn.synsets(word)

    # List to hold all hyponyms
    hypernyms = []

    # Iterate through each synset
    for synset in synsets:
        # Get hyponyms for each synset
        for hypernym in synset.hypernyms():
            # Add the lemma names (words) of each hyponym to the set
            for h in hypernym.hypernyms():
                hypernyms = hypernyms + h.lemma_names()
    words = []
    similarityies = []
    for hypernym in hypernyms:
        if(hypernym in words):
            continue
        try:
            sim = get_similarity(get_glove_embedding(hypernym), get_glove_embedding(word))
            words.append(hypernym)
            similarityies.append(sim)
        except:
            pass    
    sorted = np.argsort(similarityies)
    word_sims = []
    for idx in sorted:
        word_sims.append((words[idx], similarityies[idx]))
    for hypernym, sim in word_sims:
        if(sim > num):
            return (True, hypernym)
    return (False, "N/A")

In [10]:
def get_scale(similarity, mode):
    if mode: #True is glove
        if similarity > 0.60:
            return "hot"
        if similarity > 0.40:
            return "warm"
        else:
            return "cold"
    else:
        if similarity > 0.90:
            return "hot"
        if similarity > 0.85:
            return "warm"
        else:
            return "cold"

In [11]:
class Game():
    def __init__(self, get_embedding, hint_function):
        self.secret_word = ""
        self.get_embedding = get_embedding
        self.get_middle = hint_function
        self.guesses = 0
        self.best_sim = 0
        self.best_guess = "N/A"
        self.total_guesses = 0
    def run_game(self):
        print("This is Semantle")
        if self.get_embedding == get_GPT_embedding:
            print("Playing with GPT embeddings")
        else:
            print("Playing with gloVe")
        if self.get_middle == find_glove_similar_word:
            print("Playing with Halfway Word hints")
        elif self.get_middle == find_hypernyms:
            print("Playing with Hypernym hints")
        else:
            print("No hints!")
        print("Please guess the secret word.")
        print("Type 'give up' to end the game")
        self.secret_word = secret_word_engine()
        user_input = preprocess(input("Guess the secret word: "))
        while user_input != self.secret_word:
            self.total_guesses += 1
            try:
                if user_input == "give up":
                    print("The secret word was", self.secret_word)
                    return
                similarity = get_similarity(self.get_embedding(user_input), self.get_embedding(self.secret_word))
                scale = get_scale(similarity, self.get_embedding == get_glove_embedding)
                print(f"Similarity between {user_input} and the secret word is {similarity}, the guess is {scale}")
                if(similarity <= self.best_sim):
                    self.guesses += 1
                else:
                    self.guesses = 0
                    self.best_sim = similarity
                    self.best_guess = user_input
                if self.guesses >= 3 and self.get_middle != None:
                    self.guesses = 0
                    middle_word = self.get_middle(self.secret_word, self.best_guess, self.best_sim)
                    if(middle_word[0] == True):
                        similarity = get_similarity(self.get_embedding(middle_word[1]), self.get_embedding(self.secret_word))
                        scale = get_scale(similarity, self.get_embedding == get_glove_embedding)
                        if self.get_middle == find_glove_similar_word:
                            print("Seems like you are stuck, Here is a hint:", middle_word[1], "has the similarity of", similarity, "the hint word is", scale)
                        else:
                            print("Seems like you are stuck, Here is a hint:", middle_word[1], "is a hypernym of the target word.")
                            print("The hypernym has the similarity of", similarity, "the hint word is", scale)
                        if similarity > self.best_sim:
                                self.best_sim = similarity
                                self.best_guess = middle_word[1]
                    else:
                        print("You are close keep trying!")

            except:
                print("Sorry, try again.")
            user_input = preprocess(input("Guess the secret word: "))
        print(f"Congratulations, you won! The secret word is {self.secret_word} in {self.total_guesses}.")

In [12]:
#testing suite 1
#category 6 food
print("Test 1: select category 6; category is food")
game1 = Game(get_glove_embedding, None)
game1.run_game()

Test 1: select category 6; category is food
This is Semantle
Playing with gloVe
No hints!
Please guess the secret word.
Type 'give up' to end the game
Similarity between apple and the secret word is 0.3577006757259369, the guess is cold
Similarity between pie and the secret word is 0.6400020122528076, the guess is hot
Congratulations, you won! The secret word is cake in 2.


In [38]:
#category 7 school
print("Test 2: select category 7; category is school")
game2 = Game(get_glove_embedding, find_hypernyms)
game2.run_game()

Test 2: select category 7; category is school
This is Semantle
Playing with gloVe
Playing with Hypernym hints
Please guess the secret word.
Type 'give up' to end the game
Enter a valid number for the category
Similarity between college and the secret word is 0.3382933437824249, the guess is cold
Sorry, try again.
Similarity between university and the secret word is 0.3421485424041748, the guess is cold
Similarity between michigan and the secret word is 0.17000946402549744, the guess is cold
Similarity between secondary and the secret word is 0.13766200840473175, the guess is cold
Similarity between mascot and the secret word is 0.08379468321800232, the guess is cold
Seems like you are stuck, Here is a hint: plan is a hypernym of the target word.
The hypernym has the similarity of 0.34325147 the hint word is cold
Similarity between degree and the secret word is 0.19853974878787994, the guess is cold
Similarity between graduation and the secret word is 0.19120211899280548, the guess is c

In [39]:
#category 8 sport
print("Test 3: select category 8; category is sports")
game3 = Game(get_glove_embedding, find_glove_similar_word)
game3.run_game()

Test 3: select category 8; category is sports
This is Semantle
Playing with gloVe
No hints!
Please guess the secret word.
Type 'give up' to end the game
Similarity between score and the secret word is 0.2490340769290924, the guess is cold
Similarity between football and the secret word is 0.7896140217781067, the guess is hot
Similarity between soccer and the secret word is 0.7371647357940674, the guess is hot
Similarity between rugby and the secret word is 0.4955540895462036, the guess is warm
Similarity between goal and the secret word is 0.24878603219985962, the guess is cold
You are close keep trying!
Similarity between kicker and the secret word is 0.2383928894996643, the guess is cold
Similarity between tackle and the secret word is 0.2258613407611847, the guess is cold
Similarity between golf and the secret word is 0.4847441017627716, the guess is warm
You are close keep trying!
Similarity between ball and the secret word is 0.3575161099433899, the guess is cold
Similarity betwee